# Introduction
This notebook facilitates the manual curation of sample alignment.

In [2]:
import deltascope.alignment as ut

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

import os
import re
import time
import tqdm

# Setup

## Parameters

In [9]:
# --------------------------------
# -------- User input ------------
# --------------------------------

param = {
    'gthresh':0.5,
    'scale':[1,1,1],
    'microns':[0.16,0.16,0.21],
    'mthresh':0.5,
    'radius':10,
    'comp_order':[0,2,1],
    'fit_dim':['x','z'],
    'deg':2,
    'expname':'expname'
}

## Directories

In [ ]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Specify file paths to directories containing probability files
# after processing by ilastik
gfap = os.path.absolute('..\expname\GFAP\Prob')
at = os.path.absolute('..\expname\AT\Prob')

# Specify root directory where output will be saved
root = os.path.absolute('..\expname')

In [ ]:
# Output directory with timestamp
outname = 'Output'+time.strftime("%m-%d-%H-%M",
                                 time.localtime())

# Create output directory
outdir = os.path.join(root,outname)
os.mkdir(outdir)

## Extract list of files

In [5]:
Dat = {}
for f in os.listdir(at):
    if 'h5' in f:
        num  = re.findall(r'\d+',f.split('.')[0])[-1]
        Dat[num] = os.path.join(at,f)

In [6]:
Dzrf = {}
for f in os.listdir(gfap):
    if 'h5' in f:
        num  = re.findall(r'\d+',f.split('.')[0])[-1]
        Dzrf[num] = os.path.join(gfap,f)

In [ ]:
# Extract list of filename keys
klist = Dat.keys()

# Import raw data

In [10]:
%%time
for k in tqdm.tqdm(klist):
    if k not in list(Dbat.keys()):
        Dbat[k] = ut.preprocess(Dat[k],param)
        Dbzrf[k] = ut.preprocess(Dzrf[k],param,pca=Dbat[k].pcamed,mm=Dbat[k].mm,vertex=Dbat[k].vertex)
        #Dbcaax[k] = ut.preprocess(Dcaax[k],param,pca=Dbat[k].pcamed,mm=Dbat[k].mm,vertex=Dbat[k].vertex)
        print(k)
    else:
        print(k,'already processed')

C:\Users\zfishlab\AppData\Local\Continuum\Anaconda3\lib\site-packages\skimage\util\dtype.py:110: UserWarning: Possible precision loss when converting from float32 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


3
11
010
09
016
02
14
10
014
5
07
12
03
16
08
13
2
7
012
06
Wall time: 12min 3s


# Define experiment specific functions

Define wrapper functions for starting and saving to minimize the number of inputs that the user needs to type for each call of the function.

In [11]:
def start(k):
    return(ut.start(k,Dbat,[Dbzrf],im=True))
def save_both(k,dfa,dfb):
    ut.save_both(k,dfa,dfb,outdir,param.expname)

Save model parameters for each file to a dataframe that can be exported for later reference.

In [12]:
model = pd.DataFrame({'a':[],'b':[],'c':[]})
def save_model(k,mm,model):
    row = pd.Series({'a':mm[0],'b':mm[1],'c':mm[2]},name=k)
    model = model.append(row)
    return(model)

Define a function that can both fit a model and plot it on an existing plot

In [13]:
def fit_model(axi,df,mm=None):
    if mm == None:
        mm = np.polyfit(df.x,df.z,2)
    p = np.poly1d(mm)
    xrange = np.arange(np.min(df.x),np.max(df.x))
    axi.plot(xrange,p(xrange),c='m')
    return(mm)

Take a set of points and transform to a dataframe format for ease of access.

In [14]:
def pick_pts(x1,z1,vx,vz,x2,z2):
    pts = pd.DataFrame({'x':[x1,vx,x2],'z':[z1,vz,z2]})
    return(pts)